In [2]:
import os
from tqdm.notebook import tqdm
import pandas as pd
import file_function as ff
import re
import binascii

In [3]:
def crc32_checksum(filename):
    buf = open(filename,'rb').read()
    buf = (binascii.crc32(buf) & 0xFFFFFFFF)
    return "%08X" % buf

In [6]:
def file_info(path):

    dict_sc = {}  # 중복파일명 숫자를 카운트 할 딕셔너리
    dict_sm = {}
    dict_scm = {}

    cols = ['size', 'crc32', 'mtime',  'sell', 'root', 'stem', 'ext', 'length']
    df = pd.DataFrame(columns=cols)
    
    df_matching = pd.read_excel(
        r'D:\workspace\python_work\파일\매각사 이름매칭.xlsx')

    for root, __dirs__, files in tqdm(os.walk(path)):
        
        for f in files:
            fullname = os.path.join(root, f)
            #key
            size = str(os.path.getsize(fullname))
            #value
            mtime = str(os.path.getmtime(fullname))
            crc32 = str(crc32_checksum(fullname))
            sell = ""
            for __index__, row in df_matching.iterrows():
                if re.search(str(row[0]), str(root)):  
                    sell = row[1]
                    break            
            stem = os.path.splitext(f)[0]
            ext = os.path.splitext(f)[1]
            length = len(stem)
            
            temp1 = {"size":str(size)}
            temp2 = {"crc32": str(crc32), "mtime":str(mtime), "sell":sell, "root":root, "stem":stem, "ext":ext, "length":length}
            
            # 고유한 size, crc를 키로 하는 딕셔너리 만들기 -> mtime 다른 경우 있는지 체크
            sc = size + crc32
            if sc not in dict_sc:
                dict_sc[sc] = [temp2]
            else:
                dict_sc[sc].append(temp2)
            
            sm = size + mtime
            if sm not in dict_sm:
                dict_sc[sm] = [temp2]
            else:
                dict_sc[sm].append(temp2)

            scm = size + crc32 + mtime
            if scm not in dict_scm:
                dict_sc[scm] = [temp2]
            else:
                dict_sc[scm].append(temp2)

            # size도 하나의 col로 df 만들기
            temp1.update(temp2)
            temp_df = pd.DataFrame([temp1])
            
            df = pd.concat([df, temp_df], ignore_index=True)
    
    df.to_excel(r"C:\Users\SL\Desktop\파일정보.xlsx")
            
    return dict_sc, dict_sm, dict_scm

In [7]:
path = r'\\192.168.0.75\sollim광주\광주 공유폴더\(1.DNP_헬프)\3.채권양도통지서'
dict_sc, dict_sm, dict_scm = file_info(path)

234it [1:09:01, 17.70s/it]


FileNotFoundError: [Errno 2] No such file or directory: '\\\\192.168.10.43\\공유폴더\\@◆통합파일◆@\\(5.더키움자산관리대부_헬프)\\1.매입전 집행권원\\1901\\9008000463-1.pdf'

In [8]:
dict_sc

NameError: name 'dict_sc' is not defined